# Simulation Study

In [1]:
from preamble import *

%config InlineBackend.figure_format = 'retina'
%load_ext lab_black

In [2]:
# dill.load_session("Sim_Lognormal_Gamma.pkl")

In [3]:
import sys

print("ABC version:", abc.__version__)
print("Python version:", sys.version)
print("Numpy version:", np.__version__)
print("PyMC3 version:", pm.__version__)
print("Arviz version:", arviz.__version__)

tic()

ABC version: 0.1.1
Python version: 3.8.10 (default, May 19 2021, 18:05:58) 
[GCC 7.3.0]
Numpy version: 1.20.3
PyMC3 version: 3.11.2
Arviz version: 0.11.2


In [4]:
FAST = False

# Processor information and SMC calibration parameters
if not FAST:
    numItersData = 20
    popSize = 1000
    popSizeModels = 1000
    epsMin = 0
    timeout = 1000
else:
    numItersData = 8
    popSize = 500
    popSizeModels = 1000
    epsMin = 1
    timeout = 30

smcArgs = {"timeout": timeout, "epsMin": epsMin, "verbose": True}
smcArgs["numProcs"] = 40

## Model selection

We generate individual claim sizes from three models 

- $\text{Weib}\left(k = 1/2,\beta =  e^{1/2}/ \Gamma(3/2)\right)$
- $\text{Gamma}(r = e^{1/2},m = 1)$
- $\text{LogNormal}(\mu = 0,\sigma = 1)$

Samples of sizes $25, 50$ are considered

In [5]:
from math import gamma

np.exp(1 / 2) / gamma(3 / 2)

1.8603827342052657

In [6]:
rg = default_rng(123)

sample_sizes = [25, 50, 75, 100, 150, 200]
T = sample_sizes[-1]

claim_data = pd.DataFrame(
    {
        "lognormal": abc.simulate_claim_sizes(rg, T, "lognormal", (0, 1)),
        "gamma": abc.simulate_claim_sizes(rg, T, "gamma", (np.exp(1 / 2), 1)),
        "weibull": abc.simulate_claim_sizes(
            rg, T, "weibull", (1 / 2, np.exp(1 / 2) / gamma(3 / 2))
        ),
    }
)

## ABC posterior for choosing between lognormal, Weibull and gamma to model the claim sizes

### Lognormal data

In [7]:
models_data = ["lognormal"]
models_fitted = ["gamma", "lognormal", "weibull"]

priorG = abc.IndependentUniformPrior([(0, 5), (0, 100)], ("r", "m"))
modelG = abc.Model(sev="gamma", prior=priorG)

priorL = abc.IndependentUniformPrior([(-20, 20), (0, 5)], ("μ", "σ"))
modelL = abc.Model(sev="lognormal", prior=priorL)

priorW = abc.IndependentUniformPrior([(1e-1, 5), (0, 100)], ("k", "δ"))
modelW = abc.Model(sev="weibull", prior=priorW)

models = [modelG, modelL, modelW]

In [8]:
model_proba_abc = pd.DataFrame(
    {"model_data": [], "model_fit": [], "ss": [], "model_probability_ABC": []}
)

# model_data = "lognormal"
for model_data in models_data:
    sevs = claim_data[model_data]
    for ss in sample_sizes:
        uData = sevs[:ss]
        %time fit = abc.smc(numItersData, popSizeModels, uData, models, **smcArgs)
        for k in range(len(models)):
            weights = fit.weights[fit.models == k]
            res_mp = pd.DataFrame(
                {
                    "model_data": pd.Series(model_data),
                    "model_fit": pd.Series([models_fitted[k]]),
                    "ss": np.array([ss]),
                    "model_probability_ABC": pd.Series(
                        np.sum(fit.weights[fit.models == k])
                    ),
                }
            )
            model_proba_abc = pd.concat([model_proba_abc, res_mp])
            model_proba_abc

Final population dists <= 0.09, ESS = [250 206 359]
	model populations = [388, 228, 384], model weights = [0.46 0.17 0.37]
CPU times: user 29.9 s, sys: 2.82 s, total: 32.8 s
Wall time: 10min 31s


Final population dists <= 0.10, ESS = [222 401 196]
	model populations = [289, 492, 219], model weights = [0.33 0.5  0.17]
CPU times: user 29.4 s, sys: 1.66 s, total: 31.1 s
Wall time: 12min 29s


Final population dists <= 0.09, ESS = [ 85 750  76]
	model populations = [105, 814, 81], model weights = [0.11 0.83 0.06]
CPU times: user 28.1 s, sys: 1.95 s, total: 30.1 s
Wall time: 12min 48s


/home/plaub/miniconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Final population dists <= 0.08, ESS = [ 27 830  13]
	model populations = [39, 947, 14], model weights = [0.04 0.95 0.01]
CPU times: user 27.5 s, sys: 2.89 s, total: 30.4 s
Wall time: 14min 21s


Final population dists <= 0.09, ESS = [  5 858   1]
	model populations = [8, 990, 2], model weights = [0.01 0.99 0.  ]
CPU times: user 27 s, sys: 1.85 s, total: 28.9 s
Wall time: 8min 36s


/home/plaub/miniconda3/lib/python3.8/site-packages/approxbayescomp/smc.py:371: RuntimeWarning: invalid value encountered in double_scalars
  np.sum(weights[ms == m]) ** 2


Final population dists <= 0.07, ESS = [  0 778   0]
	model populations = [0, 1000, 0], model weights = [0. 1. 0.]
CPU times: user 25.9 s, sys: 2.04 s, total: 27.9 s
Wall time: 1min 17s


## True posterior samples

We run a Bayesian analysis on the individual claim data and compuet the model probabilities when a Weibull or a gamma distribution is assumed. The prior distribution on the parameters are taken as independent uniform distribution (as in the ABC approach). 

### Fitting a Weibull and a gamma model to the individual loss data

In [9]:
Bayesian_Summary = pd.DataFrame(
    {
        "model_data": [],
        "model_fit": [],
        "ss": [],
        "param_1": [],
        "param_2": [],
        "marginal_log_likelihood": [],
    }
)

for model_data in models_data:
    sevs = claim_data[model_data]
    for model_fitted in models_fitted:

        for ss in sample_sizes:
            uData = sevs[:ss]
            print(
                f"Fitting a {model_fitted} model to {len(uData)} data points generated from a {model_data} model"
            )

            if model_fitted == "gamma":
                with pm.Model() as model_sev:
                    r = pm.Uniform("param_1", lower=0, upper=5)
                    m = pm.Uniform("param_2", lower=0, upper=100)
                    U = pm.Gamma("U", alpha=r, beta=1 / m, observed=uData)
                    %time trace = pm.sample_smc(popSize, random_seed=1, chains=1)

            elif model_fitted == "lognormal":
                with pm.Model() as model_sev:
                    μ = pm.Uniform("param_1", lower=-20, upper=20)
                    σ = pm.Uniform("param_2", lower=0, upper=5)
                    U = pm.Lognormal("U", mu=μ, sigma=σ, observed=uData)
                    %time trace = pm.sample_smc(popSize, random_seed=1, chains=1)

            elif model_fitted == "weibull":
                with pm.Model() as model_sev:
                    k = pm.Uniform("param_1", lower=1e-1, upper=5)
                    δ = pm.Uniform("param_2", lower=0, upper=100)
                    U = pm.Weibull("U", alpha=k, beta=δ, observed=uData)
                    %time trace = pm.sample_smc(popSize, random_seed=1, chains=1)

            ll = trace.report.log_marginal_likelihood[0]

            res = pd.DataFrame(
                {
                    "model_data": [model_data],
                    "model_fit": [model_fitted],
                    "ss": [ss],
                    "param_1": [trace["param_1"].mean()],
                    "param_2": [trace["param_2"].mean()],
                    "marginal_log_likelihood": [ll],
                }
            )
            Bayesian_Summary = pd.concat([Bayesian_Summary, res])

Fitting a gamma model to 25 data points generated from a lognormal model


Initializing SMC sampler...
Sampling 1 chain in 1 job
Stage:   0 Beta: 0.010
Stage:   1 Beta: 0.047
Stage:   2 Beta: 0.129
Stage:   3 Beta: 0.277
Stage:   4 Beta: 0.779
Stage:   5 Beta: 1.000


CPU times: user 1.44 s, sys: 187 ms, total: 1.62 s
Wall time: 4.91 s
Fitting a gamma model to 50 data points generated from a lognormal model


Initializing SMC sampler...
Sampling 1 chain in 1 job
Stage:   0 Beta: 0.005
Stage:   1 Beta: 0.024
Stage:   2 Beta: 0.068
Stage:   3 Beta: 0.153
Stage:   4 Beta: 0.433
Stage:   5 Beta: 1.000


CPU times: user 1.06 s, sys: 4.45 ms, total: 1.07 s
Wall time: 1.06 s
Fitting a gamma model to 75 data points generated from a lognormal model


...

CPU times: user 1.19 s, sys: 0 ns, total: 1.19 s
Wall time: 1.19 s
Fitting a weibull model to 150 data points generated from a lognormal model


Initializing SMC sampler...
Sampling 1 chain in 1 job
Stage:   0 Beta: 0.002
Stage:   1 Beta: 0.008
Stage:   2 Beta: 0.019
Stage:   3 Beta: 0.042
Stage:   4 Beta: 0.119
Stage:   5 Beta: 0.395
Stage:   6 Beta: 1.000


CPU times: user 1.27 s, sys: 0 ns, total: 1.27 s
Wall time: 1.27 s
Fitting a weibull model to 200 data points generated from a lognormal model


Initializing SMC sampler...
Sampling 1 chain in 1 job
Stage:   0 Beta: 0.001
Stage:   1 Beta: 0.006
Stage:   2 Beta: 0.014
Stage:   3 Beta: 0.031
Stage:   4 Beta: 0.085
Stage:   5 Beta: 0.280
Stage:   6 Beta: 0.961
Stage:   7 Beta: 1.000


CPU times: user 1.57 s, sys: 0 ns, total: 1.57 s
Wall time: 1.57 s


In [10]:
Bayesian_Summary

,model_data,model_fit,ss,param_1,param_2,marginal_log_likelihood
0,lognormal,gamma,25.0,1.452940,1.338451,-44.106862
0,lognormal,gamma,50.0,1.260282,1.540875,-86.467702
0,lognormal,gamma,75.0,1.325229,1.327695,-120.298522
0,lognormal,gamma,100.0,1.404371,1.180437,-152.491596
0,lognormal,gamma,150.0,1.350877,1.294970,-235.508674
0,lognormal,gamma,200.0,1.385706,1.198068,-301.205588
0,lognormal,lognormal,25.0,0.157524,1.017816,-44.897210
0,lognormal,lognormal,50.0,0.154357,1.021445,-85.473633
0,lognormal,lognormal,75.0,0.103541,0.958989,-117.065667
0,lognormal,lognormal,100.0,0.082713,0.910577,-147.395901


In [11]:
max_marginal_log_likelihood = (
    Bayesian_Summary[["model_data", "ss", "marginal_log_likelihood"]]
    .groupby(["model_data", "ss"])
    .max()
)
max_marginal_log_likelihood.reset_index(level=["model_data", "ss"], inplace=True)
max_marginal_log_likelihood.rename(
    columns={"marginal_log_likelihood": "max_marginal_log_likelihood"}
)

Bayesian_Summary_1 = pd.merge(
    Bayesian_Summary, max_marginal_log_likelihood, how="left", on=["model_data", "ss"]
)
Bayesian_Summary_1

Bayesian_Summary_1["BF"] = np.exp(
    Bayesian_Summary_1.marginal_log_likelihood_x
    - Bayesian_Summary_1.marginal_log_likelihood_y
)

Bayesian_Summary_1
sum_BF = (
    Bayesian_Summary_1[["ss", "model_data", "BF"]].groupby(["ss", "model_data"]).sum()
)
sum_BF.reset_index(level=["model_data", "ss"], inplace=True)

Bayesian_Summary_2 = pd.merge(
    Bayesian_Summary_1, sum_BF, how="left", on=["model_data", "ss"]
)
Bayesian_Summary_2
Bayesian_Summary_2["model_probability"] = (
    Bayesian_Summary_2.BF_x / Bayesian_Summary_2.BF_y
)
Bayesian_Summary_2

,model_data,model_fit,ss,param_1,param_2,marginal_log_likelihood_x,marginal_log_likelihood_y,BF_x,BF_y,model_probability
0,lognormal,gamma,25.0,1.452940,1.338451,-44.106862,-44.106862,1.000000e+00,2.297620,4.352330e-01
1,lognormal,gamma,50.0,1.260282,1.540875,-86.467702,-85.473633,3.700679e-01,1.539944,2.403126e-01
2,lognormal,gamma,75.0,1.325229,1.327695,-120.298522,-117.065667,3.944473e-02,1.050331,3.755457e-02
3,lognormal,gamma,100.0,1.404371,1.180437,-152.491596,-147.395901,6.123047e-03,1.007013,6.080408e-03
4,lognormal,gamma,150.0,1.350877,1.294970,-235.508674,-227.963629,5.287236e-04,1.000575,5.284198e-04
5,lognormal,gamma,200.0,1.385706,1.198068,-301.205588,-290.913671,3.390605e-05,1.000035,3.390486e-05
6,lognormal,lognormal,25.0,0.157524,1.017816,-44.897210,-44.106862,4.536870e-01,2.297620,1.974595e-01
7,lognormal,lognormal,50.0,0.154357,1.021445,-85.473633,-85.473633,1.000000e+00,1.539944,6.493743e-01
8,lognormal,lognormal,75.0,0.103541,0.958989,-117.065667,-117.065667,1.000000e+00,1.050331,9.520808e-01
9,lognormal,lognormal,100.0,0.082713,0.910577,-147.395901,-147.395901,1.000000e+00,1.007013,9.930363e-01


In [12]:
# # Frequency-Loss Model
# α, p, k, β = 4, 2 / 3, 1 / 3, 1
# rg = default_rng(123)
# uData_10000 = abc.simulate_claim_sizes(rg, 10000, sev, θ_sev)
# r_mle, m_mle, BIC = infer_gamma(uData_10000, [1, 1])

# θ_plot = [[α, p, k, β], [α, p, np.NaN, np.NaN]]
# θ_mle = [[np.NaN, np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, r_mle, m_mle]]

In [13]:
model_proba = pd.merge(
    Bayesian_Summary_2[["model_data", "model_fit", "ss", "model_probability"]],
    model_proba_abc,
    how="left",
    on=["model_data", "model_fit", "ss"],
).round(2)
model_proba

,model_data,model_fit,ss,model_probability,model_probability_ABC
0,lognormal,gamma,25.0,0.44,0.46
1,lognormal,gamma,50.0,0.24,0.33
2,lognormal,gamma,75.0,0.04,0.11
3,lognormal,gamma,100.0,0.01,0.04
4,lognormal,gamma,150.0,0.00,0.01
5,lognormal,gamma,200.0,0.00,0.00
6,lognormal,lognormal,25.0,0.20,0.17
7,lognormal,lognormal,50.0,0.65,0.50
8,lognormal,lognormal,75.0,0.95,0.83
9,lognormal,lognormal,100.0,0.99,0.95


In [14]:
print(
    pd.pivot_table(
        model_proba,
        values=["model_probability", "model_probability_ABC"],
        index=["ss"],
        columns=["model_fit"],
        aggfunc={"model_probability": np.mean, "model_probability_ABC": np.mean},
    ).to_latex()
)

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{model\_probability} & \multicolumn{3}{l}{model\_probability\_ABC} \\
model\_fit &             gamma & lognormal & weibull &                 gamma & lognormal & weibull \\
ss    &                   &           &         &                       &           &         \\
\midrule
25.0  &              0.44 &      0.20 &    0.37 &                  0.46 &      0.17 &    0.37 \\
50.0  &              0.24 &      0.65 &    0.11 &                  0.33 &      0.50 &    0.17 \\
75.0  &              0.04 &      0.95 &    0.01 &                  0.11 &      0.83 &    0.06 \\
100.0 &              0.01 &      0.99 &    0.00 &                  0.04 &      0.95 &    0.01 \\
150.0 &              0.00 &      1.00 &    0.00 &                  0.01 &      0.99 &    0.00 \\
200.0 &              0.00 &      1.00 &    0.00 &                  0.00 &      1.00 &    0.00 \\
\bottomrule
\end{tabular}



In [15]:
elapsed = toc()
print(f"Notebook time = {elapsed:.0f} secs = {elapsed/60:.2f} mins")

Notebook time = 3657 secs = 60.94 mins


In [16]:
dill.dump_session("Sim_Lognormal_Gamma.pkl")